In [11]:
class MyClass():
    
    def __init__(self , name):
        print("hello world"+name)
    def yell(self):
        print("yelling")

In [14]:
x= MyClass("arbaz")

hello worldarbaz


In [15]:
x.yell()

yelling


In [21]:
class InheritedClass(MyClass):
    def __init__(self):
        super().__init__("arbaz")
        
        print("inherition")
        

In [23]:
y = InheritedClass()

hello worldarbaz
inherition


In [19]:
y.yell()

yelling


In [37]:
class Operation():
     def __init__(self,input_nodes=[]):
            self.input_nodes = input_nodes
            self.output_nodes = []
            for i in input_nodes:
                i.output_nodes.append(self)
            _default_graph.operations.append(self)
                
     def compute(self):
        pass
    


In [38]:
class add(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var + y_var
    
    
    

In [39]:
class multiply(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var * y_var
    

In [40]:
class matmul(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])
    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var.dot(y_var)
    

In [56]:
class Placeholder():
    """
    A placeholder is a node that needs to be provided a value for computing the output in the Graph.
    """
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [62]:
class Variable():
    """
    This variable is a changeable parameter of the Graph.
    """
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
         
        _default_graph.variables.append(self)

In [63]:
class Graph():
    
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        """
        Sets this Graph instance as the Global Default Graph
        """
        global _default_graph
        _default_graph = self

In [64]:
g = Graph()

In [65]:
g.set_as_default()

In [70]:
A = Variable(10)

In [71]:
B = Variable(1)

In [72]:
X = Placeholder()

In [75]:
y = multiply(A,X)

In [76]:
Z = add(y,B)

In [77]:
Z

In [78]:
def traverse_postorder(operation):
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [79]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
        
        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output




In [80]:
import numpy as np
import matplotlib.pyplot as plt
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [85]:
class Sigmoid(Operation):
 
    
    def __init__(self, z):

        # a is the input node
        super().__init__([z])

    def compute(self, z_val):
        
        return 1/(1+np.exp(-z_val))

In [82]:
g = Graph()

g.set_as_default()

A = Variable([[10,20],[30,40]])
b = Variable([1,1])

x = Placeholder()

y = matmul(A,x)

z = add(y,b)

In [83]:
s = Session()

In [84]:
s.run(z,feed_dict={x:20})

array([[201, 401],
       [601, 801]])